**Check GPU connection**


In [1]:
!nvidia-smi

Fri Apr 30 00:15:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Connect to Google Drive**

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

**Import all required libraries**


In [3]:
import random
import time
import tensorflow
import datetime
import sklearn
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop, Rescaling, Resizing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from cv2 import cv2

**Create folder for accuracy/loss charts and model summary**

In [ ]:
print("Input training name:")
training_name = input()
try:
    path = f"gdrive/MyDrive/BSL/models/{training_name}"
    !mkdir {path}
except:
    print('Path cannot be created')


In [4]:
training_name = "test" #"FINAL_BASE_MODEL"

**Convolutional Neural Network (CNN) with 4 conv layers**

In [ ]:
def train(name):
    ''' Train model function '''

    batch_size = 32
    TRAIN_PATH = "gdrive/MyDrive/BSL/FINAL_DATASET_300x300/"
    TEST_PATH = "gdrive/MyDrive/BSL/TEST_FINAL/"
    model_path = "gdrive/MyDrive/BSL/models/{}/".format(name)

    # Sets preprocessing for training images
    train_pip = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        brightness_range=[0.2, 1.0],
        rotation_range=20,
      )

    # Sets preprocessing for test images
    val_ds_pip = ImageDataGenerator(
        rescale=1./255
      )

    # Creates train dataset
    train_ds = train_pip.flow_from_directory(
        TRAIN_PATH,
        target_size = (300, 300),
        batch_size=32,
        seed=1337,
        shuffle=True,
    )

    # Creates test dataset
    val_ds = val_ds_pip.flow_from_directory(
        TEST_PATH,
        target_size = (300, 300),
        batch_size=32
    )

    # Begining of the model

    # Input layer reshapes input to be compatibile
    inputs = keras.Input(shape=(300, 300)+(3,))

    # Convolutional layer 1
    x = layers.Conv2D(128, (3, 3), padding='same', activation="relu")(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 2
    x = layers.Conv2D(128, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 3
    x = layers.Conv2D(64, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 4
    x = layers.Conv2D(64, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Dropout to prevent overfitting
    x = layers.Dropout(0.5)(x)

    # Flatten (batch, num)
    x = layers.Flatten()(x)

    # Dense layer with relu activation
    x = layers.Dense(512, activation="relu")(x)

    # Output layer
    outputs = layers.Dense(17, activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

    # Creates and saves model summary as an image
    plot_model(
        model,
        to_file=model_path + 'model_summary.png',
        show_layer_names=True,
        show_shapes=True
    )

    # Displays model summary in terminal
    model.summary()


    # Sets loss and optomazer algoriths
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    # Tensorboard
    log_dir = model_path + "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Trains the model and saves it do history variable
    history = model.fit(train_ds, batch_size=batch_size,
                        epochs=96, validation_data=val_ds, callbacks=[tensorboard_callback])

    # Evaluate
    model.evaluate(val_ds, batch_size=batch_size)


    # Saves model
    keras.models.save_model(
        model,
        model_path,
        overwrite=True,
        include_optimizer=True,
        save_format=None,
        signatures=None,
        options=None,
    )


train(training_name)
